# Cargar Datos de Agentes Libres
Este notebook se encarga actualizar la tabla con los datos de los Agentes Libres extraidos de Overthecap. Tan sólo se ejecuta cuando quieres refrescar los que han firmado/renovado o si se han borrado las tablas.

## Pre-procesado manual
Los datos se extraen manualmente de la web siguiendo este proceso:

- Ve a la web de Overthecap, selecciona toda la tabla y copia.
- Pega el contenido en Excel y guárdalo como CSV
- Desde Catalog, sube el fichero a `/Volumes/nfl-data/overthecap`

In [0]:
# File location and type
file_location = "dbfs:/Volumes/nfl-data/overthecap/2026/overthecap_freeagents_20260119.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("mergeSchema", "true") \
  .option("sep", ",") \
  .load(file_location)

#Conversiones
En esta seccion se implementan las distintas conversiones de los datos.

In [0]:
from pyspark.sql.functions import translate, col, when
from pyspark.sql.types import IntegerType, FloatType

# Rename columns with unsupported names by Delta
df = df.withColumnRenamed("Pos.", "Pos") \
       .withColumnRenamed("2025 Team", "PrevTeam") \
       .withColumnRenamed("2026 Team", "NextTeam") \
       .withColumnRenamed("Snaps", "Snaps%") \
       .withColumnRenamed("Current APY", "SalaryPY")

# Convert Vvoid contracts to UFA
df = df.withColumn('Type', when(col('Type') == 'Void', 'UFA').otherwise(col('Type')))

# Convert string currency to int
df = df.withColumn('Guarantees', translate(col("Guarantees"), "$,", "").cast(IntegerType())) \
       .withColumn('SalaryPY', translate(col("SalaryPY"), "$,", "").cast(IntegerType())) \
       .withColumn('Snaps%', translate(col("Snaps%"), "%", "").cast(FloatType()).cast(IntegerType()))

display(df)
df.printSchema()

# Actualiza Tabla
Crea la Delta table con todos los datos y ciertas conversiones que me los hacen más manejables.

In [0]:
permanent_table_name = "freeagents_2026"

df.write.format("delta").saveAsTable(permanent_table_name, mode="overwrite")

# Troubleshooting and Testing

Usa estas herramientas y comandos para comprobar el estado del workspace


In [0]:
# Delete un-used files/folders
#dbutils.fs.rm("dbfs:/FileStore/tables/nfl/pff/defense_summary.csv", True)
# List files
dbutils.fs.ls("dbfs:/Volumes/nfl-data/overthecap/2025/")

Una prueba rápida para ver que funciona la Delta Table.


In [0]:
%sql

select * from freeagents_2026 where Pos == "EDGE";